# MERGE013
Best of Each Type as of July 14

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import os
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [2]:
tracking = pd.read_csv('../tracking/tracking.csv')

In [3]:
tracking.head()

,run_id,model_number,eval_metric,n_estimators,early_stopping_rounds,random_state,n_threads,learning_rate,n_fold,model_type,...,2JHH_meta_fc_mae_cv_f1,2JHH_meta_fc_mae_cv_f2,3JHH_meta_fc_est,3JHH_meta_fc_metafolds,3JHH_meta_fc_mae_cv_f1,3JHH_meta_fc_mae_cv_f2,3JHN_meta_fc_est,3JHN_meta_fc_metafolds,3JHN_meta_fc_mae_cv_f1,3JHN_meta_fc_mae_cv_f2
0,0610_1341,M010,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0611_0308,M012,mae,500000.0,500.0,529.0,32.0,0.5,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0611_1250,M013,mae,50000.0,500.0,529.0,32.0,0.5,5.0,catboost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0611_1249,M014,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0613_0402,M015,mae,500000.0,500.0,529.0,24.0,0.3,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for cv in ['1JHC_mae_cv', '2JHH_mae_cv', '1JHN_mae_cv',
          '2JHN_mae_cv', '2JHC_mae_cv', '3JHH_mae_cv',
          '3JHC_mae_cv', '3JHN_mae_cv']:
    print(f'----------{cv}----------')
    print(tracking[['model_number','run_id','model_type', cv]] \
          .set_index('model_number') \
          .sort_values(cv) \
          .head())

----------1JHC_mae_cv----------
                 run_id model_type  1JHC_mae_cv
model_number                                   
M046_1JHC     0712_2026   catboost       0.4976
M043          0710_2124   catboost       0.5124
M045_1JHC     0711_2313   catboost       0.5244
M044          0711_1936   catboost       0.5376
M042          0709_1336   catboost       0.5650
----------2JHH_mae_cv----------
                 run_id model_type  2JHH_mae_cv
model_number                                   
M046          0714_0836   catboost       0.1251
M043          0710_2124   catboost       0.1382
M038          0707_0907   catboost       0.1405
M040          0708_1639   catboost       0.1421
M042          0709_1336   catboost       0.1428
----------1JHN_mae_cv----------
                 run_id model_type  1JHN_mae_cv
model_number                                   
M046_1JHN     0713_2329   catboost       0.3566
M030          0701_1300       lgbm       0.3582
M043          0710_2124   catboost      

In [15]:
types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC','3JHH','3JHC', '3JHN']
oof = pd.DataFrame()
sub = pd.DataFrame()
fi = pd.DataFrame()
for t in types:
    for file in os.listdir(f'../type_results/{t}/'):
        if 'M046' in file:
            if 'oof' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                oof = pd.concat([oof, df])
            if 'sub' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                sub = pd.concat([sub, df])
            if 'fi' in file:
                df = pd.read_parquet(f'../type_results/{t}/{file}')
                fi = pd.concat([fi, df])

In [27]:
for i, d in oof.groupby('type'):
    print(i)
    print(mean_absolute_error(d['scalar_coupling_constant'], d['oof_preds']))
    
glmae = group_mean_log_mae(oof['scalar_coupling_constant'], oof['oof_preds'], oof['type'])
print('\nGroup LMAE')
print(glmae)

1JHC
0.49759548437435286
1JHN
0.35656053079308153
2JHC
0.1957371996255439
2JHH
0.12512516484694158
2JHN
0.1300098094987309
3JHC
0.2216202227144778
3JHH
0.1383896179801761
3JHN
0.10535325576960344

Group LMAE
-1.6517120085012873


# Compare with Good Sub

In [19]:
sub_good = pd.read_csv('../submissions/BLEND008_submission_-1.7087CV_Jul14best.csv')

In [21]:
sub['good_scc'] = sub_good['scalar_coupling_constant']

In [29]:
sub[['scalar_coupling_constant','good_scc']].corr()

,scalar_coupling_constant,good_scc
scalar_coupling_constant,1.000000,0.999993
good_scc,0.999993,1.000000


In [39]:
sub = sub.sort_values('id').reset_index(drop=True)
oof = oof.sort_values('id').reset_index(drop=True)

In [40]:
# Save Results
sub[['id','scalar_coupling_constant']].to_csv('../submissions/MERGE013_sub_M046_-1.6517CV.csv', index=False)
oof.to_csv('../oof/MERGE013_oof_M046_-1.6517CV.csv', index=False)